# Практическое задание 4
## КМБО-01-20, Гребнев Никита, вариант 5 

Исходные данные: 
- Набор данных - [Video game sales](https://www.kaggle.com/gregorut/videogamesales)
- Тип классификатора - LogisticRegression (логистическая регрессия)
- Классификация по столбцу - Platform (PS2 – класс 0, остальные уровни – класс 1)

### Установка необходимых библиотек:

In [1]:
!pip install pandas
!pip install sklearn

### Считываем данныеиз файла, преобразуем поле Platform: `0`, если там было значение `PS2` , иначе `1`

In [2]:
import pandas
import numpy as np
data = pandas.read_csv('vgsales.csv', index_col='Rank')

# исключаем столбец Name как поле с уникальными значениями
data_sel = data.loc[:, data.columns.isin(['Platform', 'Year', 'Genre', 'Publisher', 'NA_Sales', 'EU_Sales', 'JP_Sales', 'Other_Sales'])]
data_sel = data_sel.dropna()
data_sel['Platform'] = np.where(data_sel['Platform'] == 'PS2', 0, 1)

### Разделяем данные: отдельно `Platform`, отдельно все остальные данные

In [3]:
Platform = data_sel.loc[:, data_sel.columns.isin(['Platform'])]
X = data_sel.loc[:, data_sel.columns.isin(['Year', 'Genre', 'Publisher', 'NA_Sales', 'EU_Sales', 'JP_Sales', 'Other_Sales'])]

### Преобразуем текстовые данные в числовые 

In [10]:
my_set = set(X.Genre)
i = 0
for item in my_set:
    X['Genre'] = X['Genre'].replace(item, i)
    i = i + 1
    
i = 0
my_set2 = set(X.Publisher)
for item in my_set2:
    X['Publisher'] = X['Publisher'].replace(item, i)
    i = i + 1

In [11]:
X

,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales
Rank,,,,,,,
1,2006.0,0,518,41.49,29.02,3.77,8.46
2,1985.0,6,518,29.08,3.58,6.81,0.77
3,2008.0,5,518,15.85,12.88,3.79,3.31
4,2009.0,0,518,15.75,11.01,3.28,2.96
5,1996.0,1,518,11.27,8.89,10.22,1.00
...,...,...,...,...,...,...,...
16596,2002.0,6,12,0.01,0.00,0.00,0.00
16597,2003.0,3,354,0.01,0.00,0.00,0.00
16598,2008.0,5,203,0.00,0.00,0.00,0.00


### Разделяем данные на обучающую и тестовую выборку

In [12]:
from sklearn.model_selection import train_test_split
x_train, x_validation, y_train, y_validation = train_test_split(X, Platform, test_size=.33, random_state=5)

### Выключаю предупреждения, потому что дальше их будет слишком много, и я не знаю, как их устранить 

In [13]:
import warnings
warnings.filterwarnings('ignore')

### Строим классификатор логистической регрессии

In [14]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold

grid={"C":np.logspace(-3,3,7), "penalty":["l1","l2"]}# l1 lasso l2 ridge
logreg=LogisticRegression()
logreg_cv=GridSearchCV(logreg,grid,cv=10)
logreg_cv.fit(x_train,y_train)

#logreg_cv = LogisticRegression(random_state=5).fit(x_train, y_train)

GridSearchCV(cv=10, estimator=LogisticRegression(),
             param_grid={'C': array([1.e-03, 1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02, 1.e+03]),
                         'penalty': ['l1', 'l2']})

### Оцениваем точность построенного классификатора с помощью метрик precision, recall и F1 на тестовой выборке

In [16]:
from sklearn.model_selection import cross_val_score
print("f1:"+str(np.average(cross_val_score(logreg_cv, x_validation, y_validation, scoring='f1'))))
print("precision:"+str(np.average(cross_val_score(logreg_cv, x_validation, y_validation, scoring='precision'))))
print("recall:"+str(np.average(cross_val_score(logreg_cv, x_validation, y_validation, scoring='recall'))))

f1:0.9266584270520756
precision:0.8643060170112152
recall:0.9987078245054007
